# Export simulated h for a MODFLOW 6 model with DISV grid to CSV point file 
Exports the simulated heads and create a csv file for each layer and for each time. The columns in the csv files are:
`layer,time,cell,x,y,h`<br>
These files can be imported in QGIS as csv files

In [1]:
import flopy.mf6 as mf6
import flopy.utils.binaryfile as bf
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

## Functions

In [2]:
def centroid_get(vertex_xy:[[]]):
    cx = 0.
    cy = 0.
    n = len(vertex_xy)
    for coordinate in vertex_xy:
        cx += coordinate[0]
        cy += coordinate[1]
    return cx/n, cy/n


## Set variables

In [22]:
modelname = 'sc.nam'
model_path = r'E:\LSGB\20231117_JCUNVRS\gwf\gwf4_pest'
model_path = Path(model_path)
# number of layers; if layer < 0, h will be extracted for all layeres
layer = 0
epsg_code = 25830

# head file to be read
bh_file = model_path.joinpath(modelname)
bh_file = bh_file.with_suffix('.bhd')

# output path
dir_output = r'E:\LSGB\20231117_JCUNVRS\gwf\tmp'
dir_output = Path(dir_output)


## Simulation

In [4]:
sim = mf6.MFSimulation.load(sim_name=modelname, exe_name='mf6', sim_ws=model_path)

model_names = list(sim.model_names)
print('\nModel names in sim', model_names)

gwf = sim.get_model(model_names[0])

# general information of my model
package_list = gwf.get_package_list()
print('Packages:', package_list)
nlayers = gwf.dis.nlay.array
print('Number of layers:', gwf.dis.nlay.array)  # Number of layers
ncells = gwf.dis.ncpl.array
print('Number of cells:', gwf.dis.ncpl.array)  # Number of cells per layer
ncells_per_layer = ncells / nlayers
print('Number of cells per layer:',ncells_per_layer)
ncells_per_layer = int(ncells_per_layer)
print('Number of active cells:',(gwf.dis.idomain.array > 0).sum())

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package disv...
    loading package ic...
    loading package npf...
    loading package obs...
    loading package gnc...
    loading package oc...
    loading package drn...
    loading package rch...
  loading solution package modflow...

Model names in sim ['modflow']
Packages: ['DISV', 'IC', 'NPF', 'OBS-1', 'GNC', 'OC', 'DRN-1', 'RCH-1']
Number of layers: 1
Number of cells: 2331
Number of cells per layer: 2331.0
Number of active cells: 1608


## Get the required packages

In [5]:
disv = gwf.get_package('disv')
# idomain
idomain = disv.idomain.array
# cells
cell2d = disv.cell2d.array
# vertices
vertices = disv.vertices.array
# heads
heads = bf.HeadFile(bh_file)
# get all times
times = heads.get_times()
print('Times number', len(times))
for time1 in times:
    print(time1)


Times number 1
1.0


## Writes a file for each layer and time

In [30]:
active_cells = [] # True if active, False if inactive
id_cells = [] # cells id
xyc_cells = [] # centroid coordinates of the cell as a string
for ic, cell in enumerate(cell2d):
    lcell = list(cell)
    if ic == 0:
        print('First cell:',lcell)
    id_cells.append(lcell[0])
    if idomain[layer,lcell[0]] == 1:
        active_cells.append(True)
        num_vertices = lcell[3]
        vertex_ids = lcell[4:4 + num_vertices]
        vertex_xy = [list(vertices[v1])[1:] for v1 in vertex_ids]
        xc, yc = centroid_get(vertex_xy)  
        coordinate_str = f'{xc:f},{yc:f}'
        xyc_cells.append(coordinate_str)
    else:
        active_cells.append(False)
        xyc_cells.append('')

if ncells != len(active_cells) or ncells != len(id_cells) or ncells != len(xyc_cells):
    raise ValueError(f'The number of elements in each array must be {ncells}')

if len(active_cells) % ncells_per_layer != 0:
    raise ValueError(f"ncells_per_layer ({ncells_per_layer}) must be an integer multiple of len(active_cells) ({len(active_cells)})")

active_cells_layer =  [active_cells[i:i + ncells_per_layer] for i in range(0, ncells, ncells_per_layer)]
id_cells_layer =  [id_cells[i:i + ncells_per_layer] for i in range(0, ncells, ncells_per_layer)]
xyc_cells_layer = [xyc_cells[i:i + ncells_per_layer] for i in range(0, ncells, ncells_per_layer)]

header='layer,time,cell_id,x,y,h'

if layer < 0 or layer > nlayers:
    layers = [i for i in range(nlayers)]
else:
    layers = [layer,]

for time1 in times:
    time1 = int(time1)
    for layer1 in layers:
        ofile_name = f'heads_layer_{layer1}_time_{int(time1)}.csv'
        ofile_path = dir_output.joinpath(ofile_name)
        with open(ofile_path, 'w', encoding='utf8') as fo: 
            fo.write(f'{header}\n')
            # Get the data for the specified layer and time
            heads_at_time = heads.get_data(totim=time1, mflay=layer1)
            heads_in_layer = heads_at_time[0] 
            for i, is_active in enumerate(active_cells_layer[layer1]):
                if not is_active:
                    continue
                fo.write(f'{layer1},{time1},{id_cells_layer[layer1][i]},{xyc_cells_layer[layer1][i]},{heads_in_layer[i]:f}\n')


First cell: [0, 608467.3350144, 4257604.522862, 4, 0, 1, 40, 39, None, None]
